## 🚀 Phase 1: Setup and Robot Modeling

### Step 1: Create ROS 2 Workspace and Package

In [ ]:
!mkdir -p src
!ros2 pkg create --build-type ament_python box_bot_description --dependencies rclpy sensor_msgs geometry_msgs launch_ros nav2_msgs --destination-directory src

### Step 2: Define the Robot Model (URDF)
We create the `urdf` and `world` directories, the `box_bot.urdf` and `my_world.world`files describing the physical robot and environment.

In [ ]:
!mkdir -p src/box_bot_description/urdf
!mkdir -p src/box_bot_description/world

We will run Gazobo with the custom world and spawn the robot defined in URDF file, before launch them using launch file

In [ ]:
!gz sim -r src/box_bot_description/world/my_world.world

In [ ]:
!ros2 run ros_gz_sim create -name box_bot -file src/box_bot_description/urdf/box_bot.urdf -z 0.1

We need to update `setup.py` to handle our resource files and create the simulation launch file.
Generally, every new folder added on the project should be included in setup file, to be recognaized by ROS2 during the runtime

### Step 3: Configure Gazebo and Launch Files


In [ ]:
!mkdir -p src/box_bot_description/launch

## 🚀 Phase 2: Simulation and Basic Control

### Step 4: Build and Verify Simulation

In [ ]:
!colcon build --symlink-install

### Step 5: Test Basic Movement
To test, you would open a terminal and run:


In [ ]:
!ros2 topic pub /cmd_vel geometry_msgs/msg/Twist "{linear: {x: 0.5}, angular: {z: 0.0}}"

Driving the robot Manually

In [ ]:
!ros2 run teleop_twist_keyboard teleop_twist_keyboard

## 🧠 Phase 3: Implementing Communication Mechanisms

### Step 6: Implement Core Nodes (Python) - Obstacle Avoider

Goal: To drive the robot forward indefinitely while avoiding collisions.


In [ ]:
!ros2 run box_bot_description obstacle_avoider

## 🎯 Phase 4: Navigation and Inspection Logic

### Step 8: Mapping
#### * Build a map
We will use SLAM Toolbox node to build a map of unknown environment

In [ ]:
!ros2 launch slam_toolbox online_async_launch.py use_sim_time:=True

#### * Drive Robot
We could use Teloep drinving or obstacle avoider node

#### * Saving the map

In [ ]:
!mkdir -p src/box_bot_description/maps

In [ ]:
!ros2 run nav2_map_server map_saver_cli -f /home/ubuntu/Desktop/ROS2/box_bot_ws/src/box_bot_description/maps/my_new_mapp --ros-args -p save_map_timeout:=10000.0 -p use_sim_time:=True 

### Step 9: Implement Navigation (Nav2)
We will use `nav2_bringup` package to start navigation

In [ ]:
!mkdir -p src/box_bot_description/config

In [ ]:
!ros2 launch nav2_bringup bringup_launch.py 
use_sim_time:=True autostart:=True     
map:=/home/ubuntu/Desktop/ROS2/box_bot_ws/src/box_bot_description/maps/my_new_mapp.yaml     
params_file:=/home/ubuntu/Desktop/ROS2/box_bot_ws/src/box_bot_description/config/nav2_params.yaml

This command launch bringup_launch node that contain MAp server, AMCL and Navigator nodes in the same time

- `Map server`: this node loads saved files and publish them to /map topic
- `AMCL` : localizes the robot based on the map loaded

- `The Navigation Engine`: is the collection of core servers (Planner, Controller, and Behavior Tree Navigator) that calculates the most efficient path to a goal while actively adjusting the robot's steering to avoid real-time obstacles.
